In [34]:
import fiona
from geopy.distance import distance
from pyproj import Proj, transform
from shapely.geometry import Point

#### Read .geojson data

In [14]:
locations = None  # initialize the variable since it should exist at the outermost "scope",
                  # even if Python allows you to get away with not doing so
with fiona.open('./data/location_history.geojson', 'r') as geojson_f:
    locations = [row for row in geojson_f]  # "list comprehension": a very short for loop with some benefits

#### Calculate the distance between each point

In [17]:
last_c = None
for c in locations:
    if not last_c:
        c['properties']['distanceM'] = 0.
        last_c = c
        continue
    # Note: geopy handles lat/lon natively here, but this isn't common
    c['properties']['distanceM'] = distance(last_c['geometry']['coordinates'],
                                            c['geometry']['coordinates']).meters

In [20]:
# show the distances of the first 20 points
for idx, c in enumerate(locations):
    if idx == 20:
        break
    print(c['properties']['timestamp'], c['properties']['distanceM'])

2019-05-07T16:43:48 0.0
2019-05-07T16:50:37 0.0
2019-05-07T19:37:15 60.28111706536644
2019-05-07T19:41:18 108.96574073875757
2019-05-07T19:44:09 140.46941400256958
2019-05-07T19:46:11 144.22624454671748
2019-05-07T19:56:14 164.57780690010296
2019-05-07T19:58:43 175.53619573993353
2019-05-07T20:07:34 192.91778090663803
2019-05-07T21:57:31 193.3359957549295
2019-05-08T11:25:58 198.5514840649057
2019-05-08T11:28:01 193.81766771319423
2019-05-08T11:31:34 196.19026125660835
2019-05-08T11:34:06 200.77956974908003
2019-05-08T11:36:35 209.80085850003013
2019-05-08T11:38:55 222.52867639898423
2019-05-08T11:42:18 240.32333554724062
2019-05-08T11:55:53 226.3677006293909
2019-05-08T11:58:39 208.82659964785898
2019-05-08T12:00:59 200.33942440523955


#### Buffers & Intersection

In [43]:
proj_wgs84 = Proj(init='EPSG:4326')
proj_utm18n = Proj(init='EPSG:32618')

# project lat/lon to UTM 18N and create spatial point object
first_lon, first_lat = locations[0]['geometry']['coordinates']
first_utm = transform(proj_wgs84, proj_utm18n, first_lon, first_lat)
first_point = Point(first_utm)

last_lon, last_lat = locations[-1]['geometry']['coordinates']
last_utm = transform(proj_wgs84, proj_utm18n, last_lon, last_lat)
last_point = Point(last_utm)

# create a buffer around first point and check for intersect with last point (likely False, depends on your data)
first_point_buffer = first_point.buffer(50)
first_point_buffer.intersects(last_point)

False

In [44]:
proj_wgs84 = Proj(init='EPSG:4326')
proj_utm18n = Proj(init='EPSG:32618')

# project lat/lon to UTM 18N and create spatial point object
first_lon, first_lat = locations[4]['geometry']['coordinates']
first_utm = transform(proj_wgs84, proj_utm18n, first_lon, first_lat)
first_point = Point(first_utm)

last_lon, last_lat = locations[5]['geometry']['coordinates']
last_utm = transform(proj_wgs84, proj_utm18n, last_lon, last_lat)
last_point = Point(last_utm)

# create a buffer around point and check for intersect with consecutive point (likely True, you can
# check the distanceM here)
first_point_buffer = first_point.buffer(50)
first_point_buffer.intersects(last_point)

True